Import the model that we will use in order to do image to text conversion

In [3]:
from transformers import pipeline
from PIL import Image

In [4]:
pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

Create the pipeline
pipe = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

In [5]:
# Load your image (replace 'path_to_your_image.jpg' with the actual file path)
image = Image.open('tulipe-simple-tardive-violet-beauty-2.jpg')

# Use the pipeline to generate a caption
caption = pipe(image)

# Print the generated caption
print(caption)

/home/marios/PycharmProjects/flowerProject/venv/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'purple tulips are blooming in a field of green leaves'}]
